# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [12]:
# import libraries
import joblib
import pandas as pd
import re
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sqlalchemy import create_engine
from time import time

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
# Create connection to database
engine = create_engine('sqlite:///message_features.db')
con = engine.connect()

# Find the names of tables inside of this database
engine.table_names()

['Message', 'messages']

In [14]:
# Read in data to pandas dataframe
df = pd.read_sql_table("messages", con)

df.head()

,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Create X and Y dataframes
X = df["message"]
Y = df.drop(labels=["message", "original", "genre"], axis=1)

### 2. Write a tokenization function to process your text data

In [16]:
def tokenize(text):
    
    # Normalise by setting to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # Create tokens 
    tokens = word_tokenize(text)
    
    # Remove stopwords
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    
    # Lemmatise words
    clean_tokens = [WordNetLemmatizer().lemmatize(w) for w in tokens]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [17]:
basic_logit = LogisticRegression(random_state=42)

basic_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(basic_logit, n_jobs=-1))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [18]:
# Split datasets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42)

In [19]:
# Train pipeline
b_pipeline_start_time = time()
basic_pipeline.fit(X_train, Y_train)
b_pipeline_end_time = time()

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0

In [21]:
# Print time to train
print("Time to train:", (b_pipeline_end_time - b_pipeline_start_time)/60, "mins")

NameError: name 'b_pipeline_end_time' is not defined

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [22]:
Y_pred = basic_pipeline.predict(X_test)

NotFittedError: This MultiOutputClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [23]:
for i, col in enumerate(Y.columns):
    # if i in [0, 9]: continue  # Skip bad column, TODO: Fix 0th column to not be 3 classes for no reason
    
    print("Column {}: {}".format(i, col))
    
    y_true = list(Y_test.values[:, i])
    y_pred = list(Y_pred[:, i])
    target_names = ['is_{}'.format(col), 'is_not_{}'.format(col)]
    print(classification_report(y_true, y_pred, target_names=target_names))

Column 0: related


NameError: name 'Y_pred' is not defined

### 6. Improve your model
Use grid search to find better parameters. 

In [24]:
# Define Pipeline
cv_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(OneVsRestClassifier(LogisticRegression(random_state = 42)), n_jobs=-1))
])

In [25]:
# Get parameter names
for i in cv_pipeline.get_params().keys():
    if 'clf' in i: print("\t",i)

	 clf
	 clf__estimator__estimator__C
	 clf__estimator__estimator__class_weight
	 clf__estimator__estimator__dual
	 clf__estimator__estimator__fit_intercept
	 clf__estimator__estimator__intercept_scaling
	 clf__estimator__estimator__max_iter
	 clf__estimator__estimator__multi_class
	 clf__estimator__estimator__n_jobs
	 clf__estimator__estimator__penalty
	 clf__estimator__estimator__random_state
	 clf__estimator__estimator__solver
	 clf__estimator__estimator__tol
	 clf__estimator__estimator__verbose
	 clf__estimator__estimator__warm_start
	 clf__estimator__estimator
	 clf__estimator__n_jobs
	 clf__estimator
	 clf__n_jobs


In [26]:
# Create GridSearchCV parameters
parameters = {'clf__estimator__estimator__C': [1, 2, 5],
             'clf__estimator__estimator__penalty': ['l1', 'l2']}

# Instantiate model
gridsearch_pipeline = GridSearchCV(cv_pipeline, param_grid=parameters, scoring='precision_samples', cv = 5)

In [ ]:
# Train model
gs_pipeline_start_time = time()
gridsearch_pipeline.fit(X_train, Y_train)
gs_pipeline_end_time = time()

In [ ]:
# Print time to train
print("Time to train:", (gs_pipeline_end_time - gs_pipeline_start_time)/60, "mins")

In [ ]:
gridsearch_pipeline.best_estimator_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
gridsearch_Y_Pred = gridsearch_pipeline.predict(X_test)

In [ ]:
for i, col in enumerate(Y.columns):
    # if i in [0, 9]: continue  # Skip bad column, TODO: Fix 0th column to not be 3 classes for no reason
    
    print("Column {}: {}".format(i, col))
    
    y_true = list(Y_test.values[:, i])
    y_pred = list(gridsearch_Y_Pred[:, i])
    target_names = ['is_{}'.format(col), 'is_not_{}'.format(col)]
    print(classification_report(y_true, y_pred, target_names=target_names))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
forest = RandomForestClassifier(random_state=42)

forest_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(forest, n_jobs=-1))
])

# Train pipeline
b_pipeline_start_time = time()
forest_pipeline.fit(X_train, Y_train)
b_pipeline_end_time = time()

In [ ]:
forest_Y_Pred = forest_pipeline.predict(X_test)

In [ ]:
for i, col in enumerate(Y.columns):
 
    print("Column {}: {}".format(i, col))
    
    y_true = list(Y_test.values[:, i])
    y_pred = list(forest_Y_Pred[:, i])
    target_names = ['is_{}'.format(col), 'is_not_{}'.format(col)]
    print(classification_report(y_true, y_pred, target_names=target_names))

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.